In [ ]:
import ase.build
import ase.collections
import nglview
from glob import glob
import ipywidgets as ipw
from IPython.display import clear_output

In [ ]:
mol_comp = None
def on_mol_change(c):
    global atoms, mol_comp
    atoms = ase.build.molecule(inp_mol.value)
    atoms.center(vacuum=2.0)
    atoms.pbc = True
    
    if mol_comp:
        viewer.remove_component(mol_comp)
    if surf_comp:
        viewer.remove_component(surf_comp)
    with out_cp2k:
        clear_output()
        
    mol_comp = viewer.add_component(nglview.ASEStructure(atoms)) # adds ball+stick
    viewer.add_unitcell()
    #viewer.center()

inp_mol = ipw.Dropdown(description="Molecule:", options=sorted(ase.collections.g2.names), value="H2O")
inp_mol.observe(on_mol_change, names='value')
display(inp_mol)

In [ ]:
def build_inp(atoms):
    kind_sections = ""
    for k in sorted(set(atoms.get_chemical_symbols())):
        kind_sections += "       &KIND %s\n" % k
        kind_sections += "          BASIS_SET DZVP-MOLOPT-GTH\n"
        kind_sections += "          POTENTIAL GTH-PBE\n"
        kind_sections += "       &END KIND\n"
    
    c = atoms.cell.diagonal()
    inp = """
&GLOBAL
&END GLOBAL
&FORCE_EVAL
    METHOD Quickstep
    &DFT
       BASIS_SET_FILE_NAME BASIS_MOLOPT
       POTENTIAL_FILE_NAME GTH_POTENTIALS
       &XC
          &XC_FUNCTIONAL PBE
          &END XC_FUNCTIONAL
       &END XC
       &PRINT
          &MO_CUBES
             NHOMO 5
             NLUMO 5
          &END MO_CUBES
       &END PRINT
    &END DFT
    &SUBSYS
       &CELL
          ABC %f  %f  %f
       &END CELL
       &TOPOLOGY
          COORD_FILE_NAME coords.xyz
          COORD_FILE_FORMAT XYZ
       &END TOPOLOGY
%s    &END SUBSYS
&END FORCE_EVAL
"""% (c[0], c[1], c[2], kind_sections)
    return inp

In [ ]:
def on_run_click(b):
    run_cp2k()

btn_run = ipw.Button(description="Run CP2K")
btn_run.on_click(on_run_click)
display(btn_run)

In [ ]:
out_cp2k = ipw.Output()
display(out_cp2k)

In [ ]:
def run_cp2k():
    inp = build_inp(atoms)
    open("demo.inp", "w").write(inp)
    atoms.write("coords.xyz")
    
    #TODO: remove old files
    #TODO: allow for abort
    with out_cp2k:
        clear_output()
        ! cat demo.inp
        ! cp2k.popt demo.inp

    on_cp2k_done()

In [ ]:
def on_cp2k_done():
    global inp_mo, sld_isolevel
    all_cubes = glob("*.cube")
    inp_mo = ipw.Select(options=all_cubes, height="200px")
    inp_mo.observe(on_orb_changed, "value")
    
    sld_isolevel = ipw.FloatSlider(description="Isolevel", value=0.1, min=0, max=3.0, step=0.05,
                                   continuous_update=False)
    sld_isolevel.observe(on_isolevel_change, "value")

    display(inp_mo, sld_isolevel)
    on_orb_changed(None)

In [ ]:
surf_comp = None

def on_orb_changed(c):
    global surf_comp
    if surf_comp:
        viewer.remove_component(surf_comp)
    
    fn = inp_mo.value
    surf_comp = viewer.add_component(fn, defaultRepresentation=False)
    on_isolevel_change(None)

In [ ]:
def on_isolevel_change(c):
    viewer.component_1.clear_representations()
    v = sld_isolevel.value
    viewer.component_1.add_surface(isolevel=+v, color="red")
    viewer.component_1.add_surface(isolevel=-v, color="blue")

In [ ]:
viewer = nglview.NGLWidget()
display(viewer)

In [ ]:
on_mol_change(None) # start